In [68]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_datasets as tfds

In [69]:
imdb = tf.keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [78]:
max_length = 256
embedding_dim = 16
vocab_size = 10000

In [72]:
word_index = imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}

word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [76]:
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data,
                                                           maxlen=max_length,
                                                           padding='post',
                                                           value=word_index['<PAD>'])
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data,
                                                         maxlen=max_length,
                                                         padding='post',
                                                         value=word_index['<PAD>'])

In [77]:
def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [80]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy') > 0.97):
      print('97% accuracy achieved, stopping training.')
      self.model.stop_training = True

callback = MyCallback()

In [83]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_data, train_labels,
                    epochs=10,
                    validation_data=(test_data, test_labels),
                                     callbacks=[callback])

Epoch 1/10
782/782 [==============================] - 189s 237ms/step - loss: 0.5076 - accuracy: 0.7408 - val_loss: 0.3822 - val_accuracy: 0.8439
Epoch 2/10
782/782 [==============================] - 185s 237ms/step - loss: 0.2919 - accuracy: 0.8880 - val_loss: 0.3198 - val_accuracy: 0.8749
Epoch 3/10
782/782 [==============================] - 186s 238ms/step - loss: 0.2051 - accuracy: 0.9255 - val_loss: 0.3588 - val_accuracy: 0.8585
Epoch 4/10
782/782 [==============================] - 186s 238ms/step - loss: 0.1626 - accuracy: 0.9430 - val_loss: 0.3486 - val_accuracy: 0.8560
Epoch 5/10
782/782 [==============================] - 186s 238ms/step - loss: 0.1424 - accuracy: 0.9491 - val_loss: 0.4399 - val_accuracy: 0.8554
Epoch 6/10
782/782 [==============================] - 187s 239ms/step - loss: 0.1028 - accuracy: 0.9652 - val_loss: 0.4521 - val_accuracy: 0.8572
Epoch 7/10
782/782 [==============================] - 184s 236ms/step - loss: 0.0892 - accuracy: 0.9706 - val_loss: 0.4704 -